#

# **Importing Required Libraries**

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash import Output, Input, Dash
import dash_bootstrap_components as dbc
import plotly.express as px
from urllib.request import urlopen
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import json
import shapely
from shapely.geometry import Point, Polygon
from dash import State
import re
import datetime
from datetime import date, datetime, timedelta
import requests

C:\Users\Owner\AppData\Local\Temp\ipykernel_18188\4222871443.py:6: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\Owner\AppData\Local\Temp\ipykernel_18188\4222871443.py:7: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


# **DASH APP**

In [39]:
masterDf = pd.read_csv('CMPD_cleaned.csv')

#opening new filtered GeoJSON for later use
with open('charlotte_zip_json', 'r') as f:
    zip_codes = json.load(f)

masterDf['DATE_REPORTED'] = pd.to_datetime(masterDf['DATE_REPORTED'])

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP], suppress_callback_exceptions=True)
px.set_mapbox_access_token('pk.eyJ1IjoiYWpub3NpbXAiLCJhIjoiY2x1amN0MXQxMDh0azJtcjBlbGNkM3N1ZiJ9.l0b4gnsmLgX8o-ysOSEzCw')

# styling the sidebar
SIDEBAR_STYLE = {
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "16rem",
    "padding": "2rem 1rem",
    "background-color": "#2E2C2C",
}

# padding for the page content
CONTENT_STYLE = {
    "margin-left": "16rem",
    "margin-right": "2rem",
    "margin-top": "0rem",
    "padding": "1rem 5rem",
    "background-color": "#111111"
}

#Making the sidebar element of page
sidebar = html.Div(
    [
        html.H2("CMPD Dashboard", className="display-5", style={'color':'white'}),
        html.Hr(),
        #Dcc dropdown that allows users to filter figures based on a certain crime reported
        dcc.Dropdown(id='crime_dropdown', options = sorted(masterDf['HIGHEST_NIBRS_DESCRIPTION'].unique()),
                 optionHeight = 35,
                 searchable = True,
                 clearable=True,
                 style={'margin-top': '10px', 'background-color':'black'},
                 placeholder = 'Select a crime...'
        ),
        html.P(
            "Based on CMPD data from 2017-2024", className="lead", style={'margin-top': '5px', "color": 'white'}
        ),
        #Establishes the link names of the other poges accesible by sidebar
        dbc.Nav(
            [
                dbc.NavLink("Incidents Reported", href="/", active="exact"),
                dbc.NavLink("Incidents by Day", href="/page-1", active="exact"),
                dbc.NavLink("Incidents over Time", href="/page-2", active="exact"),
            ],
            vertical=True,
            pills=True,
        )
    ],
    style=SIDEBAR_STYLE
)

content = html.Div(id="page-content", children=[], style=CONTENT_STYLE)

app.layout = html.Div([
    dcc.Location(id="url"),
    sidebar,
    content
])


#This callback updates the graphs on the home page based on the dropdown value, the date collected from hovering over line graph, and the zip code slected from choropleth map
@app.callback(
    [Output(component_id='choropleth_map', component_property = 'figure'), 
     Output(component_id='line_graph', component_property = 'figure'),
     Output(component_id='bar_graph', component_property='figure'),
     Output(component_id='pie-chart', component_property='figure'),
     Output(component_id='total_card', component_property='children'),
     Output(component_id='most-reported-card', component_property='children')],
    [Input(component_id='crime_dropdown', component_property='value'),
     Input(component_id='line_graph', component_property='hoverData' ),
     Input(component_id='choropleth_map', component_property='clickData')]
)
def update_graphs_on_p1(val_chosen, hov_data, click_data):

    
    filtered_df = masterDf.copy()
    TrendsDf = masterDf.copy()

    if not hov_data:
        #if no hover data and no dropdown value, no filters needed
        if val_chosen:
            #case of dropdown value only
            filtered_df = filtered_df[filtered_df['HIGHEST_NIBRS_DESCRIPTION']==val_chosen]
            TrendsDf = TrendsDf[TrendsDf['HIGHEST_NIBRS_DESCRIPTION']==val_chosen]
            #case of drowdown value and click data
            if click_data:
                 filtered_df = filtered_df[filtered_df['ZIP']==click_data['points'][0]['location']]
                 TrendsDf = TrendsDf[TrendsDf['ZIP']==click_data['points'][0]['location']]
        #case of only click data
        elif click_data:
            filtered_df = filtered_df[filtered_df['ZIP']==click_data['points'][0]['location']]
            TrendsDf = TrendsDf[TrendsDf['ZIP']==click_data['points'][0]['location']]
            
    if hov_data:
        #checks for dropdown data first then filters based on hover data
        if val_chosen:
            #case of dropdown value and hover data
            filtered_df = filtered_df[filtered_df['HIGHEST_NIBRS_DESCRIPTION']==val_chosen]
            TrendsDf = TrendsDf[TrendsDf['HIGHEST_NIBRS_DESCRIPTION']==val_chosen]
            #case of click data hover data and dropdown data
            if click_data:
                filtered_df = filtered_df[filtered_df['ZIP']==click_data['points'][0]['location']]
                TrendsDf = TrendsDf[TrendsDf['ZIP']==click_data['points'][0]['location']]
        #case of click data and hover data
        elif click_data:
            filtered_df = filtered_df[filtered_df['ZIP']==click_data['points'][0]['location']]
            TrendsDf = TrendsDf[TrendsDf['ZIP']==click_data['points'][0]['location']]
                
            
        filtered_df = filtered_df[(filtered_df['MONTH_NAME']==hov_data['points'][0]['x'][0:3]) & (filtered_df['YEAR']==int('20' + hov_data['points'][0]['x'][-2:]))]

    
    choropleth_df = filtered_df[filtered_df['ZIP']!='nan']
    choropleth_df = pd.DataFrame(choropleth_df['ZIP'].value_counts())
              
    
    zipMap = px.choropleth_mapbox(choropleth_df,
                                  geojson=zip_codes,
                                  locations=choropleth_df.index,
                                  featureidkey="properties.ZCTA5CE10",
                                  color='count',
                                  color_continuous_scale="BuPu",
                                  zoom=8.7,
                                  center = {"lat": 35.227085, "lon": -80.853124},
                                  mapbox_style='dark',
                                  opacity=0.3,
                                  labels={'count':'Incidents'},
                                  height=350, width=450)

    zipMap.update_layout(margin={"r":0,"t":0,"l":0,"b":0},
                         coloraxis_colorbar_bgcolor="black",
                         coloraxis_colorbar_bordercolor="black",
                         coloraxis_colorbar_tickfont_color="white",
                         coloraxis_colorbar_title_font_color="white",
                         coloraxis_colorbar_x=0.0
    )
    

    #Grouping DataFrame by month and year to count # rows in each month
    TrendsDf = TrendsDf.groupby(by=['YEAR', 'MONTH_NAME']).count()

    #I only want months where there is crimes reported, im arbitrarily choosing the DAY_WEEK column to act as a "count" column
    TrendsDf = TrendsDf[TrendsDf['DAY_WEEK']>0]
    
    TrendsDf['COUNT'] = TrendsDf['DAY_WEEK']
    TrendsDf = TrendsDf[['COUNT']]

    #Adding a column that puts date of Month and Year in format 'Jan '18'
    MONTH_YEAR = []
    for date in TrendsDf.index:
        MONTH_YEAR.append(date[1] + " " + "'" + str(date[0])[-2:])
    TrendsDf['MONTH_YEAR'] = MONTH_YEAR


    lineGraph = px.line(data_frame=TrendsDf,
                  x= 'MONTH_YEAR',
                  y='COUNT',
                  labels = {'COUNT':'# of Incidents', 'MONTH_YEAR':'Month'},
                  title = 'Number of Incidents Reported',
                  template = 'plotly_dark'
    )
    lineGraph.update_traces(mode="markers+lines", hovertemplate=None)
    lineGraph.update_layout(hovermode="x unified",
                      width=1400, height = 425,
                      margin=dict(b=80,l=80,r=10,t=80)
    )
    lineGraph.update_xaxes(tickangle=60,
                     tickmode='array',
                     tickvals = MONTH_YEAR[::6]
    )
   

    if not val_chosen:
        val_chosen = ""
    #filters dataframe and draws bar graph
    BarGraph = px.bar(data_frame = filtered_df['CMPD_PATROL_DIVISION'].value_counts(),
                      y = filtered_df['CMPD_PATROL_DIVISION'].value_counts().index,
                      x = 'count',
                      labels = {'count':'# of Incidents'},
                      template = 'plotly_dark',
                      orientation = 'h',
                      color_discrete_sequence = ['MediumVioletRed'],
                      height=350, width=650)
    
    BarGraph.update_layout(
        margin={"r":50,"t":50,"l":50,"b":50},
        title = {'text': f'{val_chosen} Incidents by CMPD District', 'x': 0.5, 'xanchor': 'center', 'yanchor': 'top'},
        yaxis=dict(title='')
        )

    BarGraph.update_xaxes(tickangle=60)


    pie_df = masterDf['DAY_WEEK'].value_counts()
    pie_df = pd.DataFrame(pie_df)
    pie_df.reset_index(drop=False, inplace=True)


    pie_chart = px.pie(
        data_frame = pie_df,
        names = 'DAY_WEEK',
        values='count',
        height = 450,
        width = 425,
        hole= 0.2,
        title='Day of Week of Crimes',
        color_discrete_sequence = ['BlueViolet', 'Indigo', 'MediumSlateBlue', 'MidnightBlue', 'Orchid', 'MediumVioletRed'],
        labels = {'DAY_WEEK': 'Day of Week', 'count':'# Incidents'}
    )
    
    pie_chart.update_layout(
        title = {'text': 'Day of Week of Crimes', 'x': 0.5, 'xanchor': 'center', 'yanchor': 'top'},
        showlegend=False,
        paper_bgcolor='#111111',
        font_color='white'
    )
    pie_chart.update_traces(textinfo='label+percent', 
                      insidetextfont={'color':'white'})

    
    TotalCardVal = html.Div([
                html.H5('Total Incidents', style={'textAlign':'center'}),
                html.H6("{:,}".format(filtered_df.shape[0]), style={'textAlign': 'center'})     
            ])
    
    #If statment shortens the crime description if it is longer than 13 characters
    if len(filtered_df['HIGHEST_NIBRS_DESCRIPTION'].mode()[0]) > 13:
        MostReportedCardVal = html.Div([
            html.H6('Most Reported Incident', style={'textAlign':'center'}),
            html.H5(filtered_df['HIGHEST_NIBRS_DESCRIPTION'].mode()[0][:13]+"...", style={'textAlign': 'center'})     
        ])
    elif len(filtered_df['HIGHEST_NIBRS_DESCRIPTION'].mode()[0]) <= 13:
        MostReportedCardVal = html.Div([
            html.H6('Most Reported Incident', style={'textAlign':'center'}),
            html.H5(filtered_df['HIGHEST_NIBRS_DESCRIPTION'].mode()[0], style={'textAlign': 'center'})     
        ])
        
    return zipMap, lineGraph, BarGraph, pie_chart, TotalCardVal, MostReportedCardVal


#this callback updates the graphs on the second page taking the dropdown value and date selected as inputs.
@app.callback(
     [Output(component_id='scatter-map', component_property='figure'),
      Output(component_id="indicator-card", component_property='figure'),
      Output(component_id='day-bar-graph', component_property='figure'),
      Output(component_id='district-card', component_property='children')], 
     [Input(component_id='crime_dropdown', component_property='value'),
     Input(component_id='date-picker', component_property='date' )]
)
def page_2_updates(val_chosen, date_chosen):
    
    filtered_df = masterDf.copy()
    day_before_df = masterDf.copy()

    #filtering dataframes based on input
    if val_chosen:
        filtered_df = filtered_df[filtered_df['HIGHEST_NIBRS_DESCRIPTION']==val_chosen]
        
        if date_chosen:
            filtered_df = filtered_df[filtered_df['DATE_REPORTED']==pd.to_datetime(date_chosen)]
            day_before_df = day_before_df[day_before_df['HIGHEST_NIBRS_DESCRIPTION']==val_chosen]
            day_before = pd.to_datetime(date_chosen) - pd.Timedelta(days=1)
            day_before_df = masterDf[masterDf["DATE_REPORTED"]==day_before]
            
    elif date_chosen:

        filtered_df = filtered_df[filtered_df['DATE_REPORTED']==pd.to_datetime(date_chosen)]
        day_before = pd.to_datetime(date_chosen) - pd.Timedelta(days=1)
        day_before_df = day_before_df[day_before_df["DATE_REPORTED"]==day_before]
        
    
    scat_map = px.scatter_mapbox(
        data_frame = filtered_df,
        lat = filtered_df["LATITUDE_PUBLIC"],
        lon = filtered_df["LONGITUDE_PUBLIC"],
        hover_data={'HIGHEST_NIBRS_DESCRIPTION': True, 'DATE_REPORTED': True, 'LATITUDE_PUBLIC': False, 'LONGITUDE_PUBLIC': False },
        mapbox_style = 'dark',
        center = {"lat": 35.227085, "lon": -80.843124},
        zoom=10,
        labels = {'HIGHEST_NIBRS_DESCRIPTION': 'Crime', 'DATE_REPORTED': 'DATE'}   
    )
    
    scat_map.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, width=710, height=800)
    
    scat_map.update_traces(
        marker=dict(
            size=6,  # Specify the size of markers
            color='MediumVioletRed'  # Specify the color of markers
        )
    )

    if date_chosen:
        day_card = go.Figure(go.Indicator(
            mode = "number+delta",
            value = filtered_df.shape[0],
            delta = {'position': "top", 'reference': day_before_df.shape[0]},
            domain = {'x': [0, 1], 'y': [0, 1]},
            title = {"text": "Total Incidents (from Day Before)", "font": {"color": "white", "size": 12}},
            number_font={'size': 40, 'color':'white'}))
        
        day_card.update_layout(paper_bgcolor = "black",
                          height = 125, width=250)

    elif not date_chosen:
        day_card = go.Figure(go.Indicator(
            mode = "number",
            value = filtered_df.shape[0],
            domain = {'x': [0, 1], 'y': [0, 1]},
            title = {"text": "Total Incidents (from Day Before)", "font": {"color": "white", "size": 12}},
            number_font={'size': 40, 'color':'white'}))
        
        day_card.update_layout(paper_bgcolor = "black",
                          height = 125, width=250)

    if filtered_df.shape[0] > 0:
        
        district_card = html.Div([
                html.H6('District with Most Incidents', style={'textAlign':'center'}),
                html.H2(filtered_df['CMPD_PATROL_DIVISION'].value_counts().index[0], style={'textAlign': 'center'})     
            ])

    elif filtered_df.shape[0] == 0:
        
        district_card = html.Div([
                html.H1('No Incidents', style={'textAlign': 'center'})     
            ])

    
    bar_graph = px.bar(data_frame = filtered_df['HIGHEST_NIBRS_DESCRIPTION'].value_counts().sort_values(ascending = False).head(10),
                       y=filtered_df['HIGHEST_NIBRS_DESCRIPTION'].value_counts().sort_values(ascending = False).head(10).index,
                       x='count',
                       template='plotly_dark',
                       labels={'count':'# of Incidents',},
                       orientation='h',
                       color_discrete_sequence=['MediumVioletRed'],
                       height=400, width=700)

    bar_graph.update_layout(yaxis=dict(autorange='reversed', title=''),
                            title=f'10 Most Common crimes on {pd.to_datetime(date_chosen).strftime('%B %d, %Y')}', 
                            title_x=0.5)
                         
    return scat_map, day_card, bar_graph, district_card

#this callback allows the choropleth map to be unfiltered by clicking a button on the page
@app.callback(
    Output('choropleth_map', 'clickData'),
    [Input('clear-button', 'n_clicks')]
)
def clear_filter(n_clicks):
    # If button is clicked, clear filter by setting clickData to None
    if n_clicks is not None and n_clicks > 0:
        return None
    else:
        # Return existing clickData if button is not clicked
        return dash.no_update


@app.callback(
     Output(component_id='choropleth_map_animation', component_property='figure'), 
     Input(component_id='crime_dropdown', component_property='value')
)

def page_3_updates(val_chosen):

    filtered_df = masterDf.copy()
    
    if val_chosen:
        #filter the dataframe based on the crime selected
        filtered_df[filtered_df['HIGHEST_NIBRS_DESCRIPTION']==val_chosen]


    filtered_df = filtered_df.groupby(by=['YEAR', 'MONTH_NAME', 'ZIP']).count()

    #Once again using an arbitrary column with no null values to make a Count column for groupby
    filtered_df['COUNT'] = filtered_df[['DAY_WEEK']]
    filtered_df = filtered_df['COUNT']
    filtered_df = filtered_df.reset_index()
    
    
    def month_year(row):
        return row['MONTH_NAME'] + " '" + str(row['YEAR'])[-2:]
    
    filtered_df['MONTH_YEAR'] = filtered_df.apply(month_year, axis=1)
    
    animated_map = px.choropleth_mapbox(filtered_df, geojson=zip_codes, locations='ZIP', featureidkey="properties.ZCTA5CE10", color='COUNT',
                                  color_continuous_scale="BuPu",
                                  zoom=10, center = {"lat": 35.227085, "lon": -80.843124},
                                  mapbox_style='dark',
                                  opacity=0.3,
                                  labels={'COUNT':'Incidents'},
                                  animation_frame ='MONTH_YEAR',
                                  animation_group='ZIP',
                                  height=850, width=1450)
    
    animated_map.update_layout(margin={"r":0,"t":0,"l":0,"b":0},
                         coloraxis_colorbar_bgcolor="black",
                         coloraxis_colorbar_bordercolor="black",
                         coloraxis_colorbar_tickfont_color="white",
                         coloraxis_colorbar_title_font_color="white",
                         coloraxis_colorbar_x=0.0
    )


    animated_map.update_layout(
    sliders=[
        dict(
            active=0,
            visible=True,
            pad=dict(t=20, b=10),
            len=0.9,
            y=0,
            bgcolor="black"  # Color of the slider track (timeline part)
        )
        ]
    )
    return animated_map
            


#This callback works with the sidebar and displays information depending on what pathname is selected
@app.callback(
    Output("page-content", "children"),
    [Input("url", "pathname")]
)
def render_page_content(pathname):
    #homepage
    if pathname == "/":
        return [
            html.Div(style={
                'position': 'fixed',
                'top': 0, 'left': 256, "right":0,
                'width': '100%',
                'height': '100%',
                'backgroundColor': '#111111'}
            ),
            html.Div([
                html.H1(
                    'CMPD Incidents Reported', 
                    style={'color': 'white', 'textAlign': 'center'})],
                    style={'position': 'relative', 'left': '50%', 'transform': 'translateX(-50%)'}),
            html.Div([
                dbc.Card(
                    dbc.CardBody(
                        id='total_card',
                        children={}, style={'color':'white'}), 
                        style={'position':'absolute', 'left': 750, 'top': 85, 'backgroundColor': 'black', 'width': "16rem", 'height':'5.5rem'}
            )]),
            html.Div([
                dbc.Card(
                    dbc.CardBody(
                        id='most-reported-card',
                        children={}, style={'color':'white'}), 
                        style={'position':'absolute', 'left': 1075, 'top': 85, 'backgroundColor': 'black', 'width': "16rem", 'height':'5.5rem'}
            )]),
            html.Div([html.Button("Clear Filter", id="clear-button")], style={'position': 'absolute', 'top': 175, 'left': 395, 'width': '0%', 'height': '0%'}),
            html.Div([
                dcc.Graph(id='choropleth_map', 
                          figure={})],
                          style={'position': 'absolute', 'top': 200, 'left': 395, 'width': '0%', 'height': '0%'}),
            html.Div([
                dcc.Graph(id='pie-chart', 
                          figure={})],
                          style={'position': 'absolute', 'top': 175, 'left': 820, 'width': '0%', 'height': '0%'}),
            html.Div([
                dcc.Graph(id='bar_graph',figure={})],
                style={'position': 'absolute', 'top': 225, 'left': 1170, 'width': '0%', 'height': '0%'}),
            html.Div([
                dcc.Graph(id='line_graph',
                          figure={},
                          clear_on_unhover=True, 
                          style={'width': '90vh', 'height': '90vh'})],
                style={'position': 'absolute', 'top': 550, 'left': 370, 'width': '0%', 'height': '0%'}),
            html.H2("*~1.2% of incidents not reflected in choropleth map due to unrecoverable null zip code values in dataset", 
                    style={'position':'absolute', 'top':990, 'left':370, 'color':'white', 'font-size':'14px'}),
            html.H2("*Dataset only reflects incidents responded to by CMPD, does not include incidents from Concord police, Mint Hill police, etc", 
                    style={'position':'absolute', 'top':1005, 'left':370, 'color':'white', 'font-size':'14px'})
        ]

    elif pathname == "/page-1":
        #there is a glitch where jupyter 'forgets' that i have imported date causing errors, so i import it before it is used in callback
        from datetime import date
        return [
            html.Div(style={
            'position': 'fixed',
            'top': 0, 'left': 256,
            'width': '100%',
            'height': '100%',
            'backgroundColor': '#111111'}
        ),
        html.Div([
            html.H1(
                "Incidents by Day", 
                style={'color': 'white', 'textAlign': 'center'})],
                style={'position': 'relative', 'left': '50%', 'transform': 'translateX(-50%)'}),
        html.Hr(style={'color': 'white', 'height': '15px'}),
        html.Div([
                dcc.DatePickerSingle(
                    id='date-picker',
                    clearable=True,
                    min_date_allowed=date(2016, 12, 31),
                    max_date_allowed=date(datetime.now().year, datetime.now().month, datetime.now().day),
                    initial_visible_month=date(2023, 8, 5),
                    display_format='YYYY-MM-DD',
                    date=date(2020, 8, 25),
                    style={'width': "100%", 'height': "100%"})],
                style={'position': 'absolute', 'left': 1400, 'top': 175}),
        html.Hr(style={'position': 'absolute', 'left': 1100, 'top': 265, 'borderColor': 'white', 'borderWidth': '5px', 'height': '0px', 'width':'37%'}),
        html.Div([
            dcc.Graph(id='scatter-map',
                 figure={})],
            style={'position': 'absolute', 'left':335, 'top':100, 'width':'5%', 'height':'10%'}),
        html.Div([
            dcc.Graph(id='indicator-card',
                 figure={})],
            style={'position': 'absolute', 'left':1500, 'top':325, 'width':'5%', 'height':'0%'}),
        html.Div([
                dbc.Card(
                    dbc.CardBody(
                        id='district-card',
                        children={}, style={'color':'white', 'position': 'absolute', 'top':10}),
                        style={'position':'absolute', 'left': 1200, 'top': 325, 'backgroundColor': 'black', 'width': "250px", 'height':'125px', "borderRadius": "0"}
            )]),
        html.Hr(style={'position': 'absolute', 'left': 1100, 'top': 475, 'borderColor': 'white', 'borderWidth': '5px', 'height': '0px', 'width':'37%'}),
        html.Div([
        dcc.Graph(id='day-bar-graph',
             figure={})],
        style={'position': 'absolute', 'left':1125, 'top':525, 'width':'5%', 'height':'1%'})
        ]
    elif pathname == "/page-2":
        return [
             html.Div(style={
            'position': 'fixed',
            'top': 0, 'left': 256,
            'width': '100%',
            'height': '100%',
            'backgroundColor': '#111111'}
        ),
        html.Div([
            html.H1(
                "Incidents over Time", 
                style={'color': 'white', 'textAlign': 'center'})],
                style={'position': 'relative', 'left': '50%', 'transform': 'translateX(-50%)'}),
        html.Div([
            dcc.Graph(id='choropleth_map_animation',
                 figure={})],
            style={'position': 'absolute', 'left':335, 'top':100, 'width':'5%', 'height':'10%'})
        ]
        
    # If the user tries to reach a different page, return a 404 message
    return dbc.Jumbotron(
        [
            html.H1("404: Not found", className="text-danger"),
            html.Hr(),
            html.P(f"The pathname {pathname} was not recognised..."),
        ]
    )

app.run(jupyter_mode="external")

if __name__=='__main__':
    app.run_server(debug=True, port=3000)


Dash app running on http://127.0.0.1:8050/


In [31]:
masterDf.head()

,Unnamed: 0,YEAR,INCIDENT_REPORT_ID,LOCATION,ZIP,LATITUDE_PUBLIC,LONGITUDE_PUBLIC,DIVISION_ID,CMPD_PATROL_DIVISION,NPA,...,PLACE_DETAIL_DESCRIPTION,CLEARANCE_STATUS,CLEARANCE_DATE,HIGHEST_NIBRS_CODE,HIGHEST_NIBRS_DESCRIPTION,OBJECTID,TIME_TO_CLEAR,MONTH,MONTH_NAME,DAY_WEEK
0,0,2023,20230610-1356-01,5700 N TRYON ST,28213.0,35.262857,-80.772157,07,North Tryon,371,...,Other - Commercial Place,Open,NaN,240,Motor Vehicle Theft,1,NaN,6,Jun,Saturday
1,1,2023,20230615-2345-01,400 HILO DR,28206.0,35.257212,-80.806051,07,North Tryon,21,...,Apartment/Duplex Private Res,Open,NaN,13B,Simple Assault,2,NaN,6,Jun,Thursday
2,2,2021,20211001-0524-01,5500 PANORAMA AV,28213.0,35.282006,-80.766679,07,North Tryon,103,...,Apartment/Duplex Private Res,Open,NaN,23F,Theft From Motor Vehicle,3,NaN,10,Oct,Friday
3,3,2019,20190829-1402-04,300 BRADFORD DR,28208.0,35.258187,-80.897111,27,Freedom,385,...,Beauty Salon/Barber Shop,Open,NaN,290,Damage/Vandalism Of Property,4,NaN,8,Aug,Thursday
4,4,2022,20221205-1912-01,7400 E INDEPENDENCE BV,28227.0,35.162612,-80.739408,17,Independence,186,...,Other - Retail,Open,NaN,23C,Shoplifting,5,NaN,12,Dec,Monday


In [14]:
TrendsDf = masterDf.copy()

TrendsDf = TrendsDf.groupby(by=['YEAR', 'MONTH_NAME']).count()

#I only want months where there is crimes reported, im arbitrarily choosing the DAY_WEEK column to act as a "count" column
TrendsDf = TrendsDf[TrendsDf['DAY_WEEK']>0]

TrendsDf['COUNT'] = TrendsDf['DAY_WEEK']
TrendsDf = TrendsDf[['COUNT']]
TrendsDf

COUNT
YEAR MONTH_NAME       
2017 Apr          5951
     Aug          6744
     Dec          6101
     Feb          5797
     Jan          6465
...                ...
2023 Sep          6573
2024 Apr          5786
     Feb          5985
     Jan          6496
     Mar          6029

[88 rows x 1 columns]

In [29]:
pie_df = masterDf['DAY_WEEK'].value_counts()
pie_df = pd.DataFrame(pie_df)
#pie_df['count'] = pie_df
pie_df.reset_index(drop=False, inplace=True)
pie_df

,DAY_WEEK,count
0,Tuesday,81182
1,Monday,81090
2,Wednesday,80924
3,Thursday,80077
4,Friday,78928
5,Saturday,67656
6,Sunday,66633


In [37]:
masterDf.dtypes

Unnamed: 0                     int64
YEAR                           int64
INCIDENT_REPORT_ID            object
LOCATION                      object
ZIP                          float64
LATITUDE_PUBLIC              float64
LONGITUDE_PUBLIC             float64
DIVISION_ID                   object
CMPD_PATROL_DIVISION          object
NPA                            int64
DATE_REPORTED                 object
DATE_INCIDENT_BEGAN           object
DATE_INCIDENT_END             object
PLACE_TYPE_DESCRIPTION        object
PLACE_DETAIL_DESCRIPTION      object
CLEARANCE_STATUS              object
CLEARANCE_DATE                object
HIGHEST_NIBRS_CODE            object
HIGHEST_NIBRS_DESCRIPTION     object
OBJECTID                       int64
TIME_TO_CLEAR                 object
MONTH                          int64
MONTH_NAME                    object
DAY_WEEK                      object
dtype: object